# 🔌CONEXÃO
Estabelecer conexão com os bancos

In [4]:
import psycopg2
from psycopg2.extras import DictCursor
import fdb
from dotenv import load_dotenv
import os

load_dotenv()

# Conexões
cnx_dest = fdb.connect(
    user=os.getenv("FDB_USER"),
    password=os.getenv("FDB_PASS"),
    host=os.getenv("FDB_HOST"),
    port=int(os.getenv("FDB_PORT")),
    database=os.getenv("FDB_PATH"),
    charset="WIN1252"
)
cur_dest = cnx_dest.cursor()


cnx_orig = psycopg2.connect(
    user=os.getenv("PG_USER"),
    password=os.getenv("PG_PASS"),
    host=os.getenv("PG_HOST"),
    database=os.getenv("PG_DB"),
    options="-c search_path={}".format(os.getenv("PG_SCHEMA"))
)
cnx_orig.autocommit = True
cur_orig = cnx_orig.cursor(cursor_factory=DictCursor)

def commit():
    cnx_dest.commit()

# 🛠️ FERRAMENTAS
Funções, variáveis cache, hashmaps

In [5]:
global cadest, empresa, exercicio
cadest = {}
empresa = cur_dest.execute("SELECT empresa FROM cadcli").fetchone()[0]
exercicio = '2025' #cur_dest.execute("SELECT mexer FROM cadcli").fetchone()[0]

def limpa_tabela(tabelas):
    for tabela in tabelas:
        cur_dest.execute(f"DELETE FROM {tabela}")
    commit()

def cria_coluna(tabela, coluna):
    try:
        cur_dest.execute(f"ALTER TABLE {tabela} ADD {coluna} VARCHAR(255)")
    except fdb.DatabaseError as e:
        print(f"Erro ao criar coluna {coluna} na tabela {tabela}: {e}")
    else:
        commit()

def cria_coluna(tabela, coluna):
    try:
        cur_dest.execute(f"ALTER TABLE {tabela} ADD {coluna} VARCHAR(255)")
    except fdb.DatabaseError as e:
        print(f"Erro ao criar coluna {coluna} na tabela {tabela}: {e}")
    else:
        commit()

def to_cp1252_safe(value, replace_with=''):
    """
    Converte uma string para cp1252 de forma segura.
    Remove ou substitui caracteres inválidos.

    :param value: valor a converter
    :param replace_with: string usada no lugar de caracteres inválidos (default = '')
    :return: valor limpo
    """
    if isinstance(value, str):
        try:
            # Tenta converter diretamente
            value.encode('cp1252')
            return value
        except UnicodeEncodeError:
            # Remove/substitui caracteres inválidos
            return value.encode('cp1252', errors='replace').decode('cp1252').replace('?', replace_with)
    return value

cur_dest.execute("SELECT codreduz, max(cadpro) FROM cadest group by 1")
if cur_dest.description is None:
    print("CADEST VAZIA!")
else:
    cadest = {k:v for k,v in cur_dest.execute("SELECT codreduz, max(cadpro) FROM cadest group by 1").fetchall()}


# CADASTROS

## TIPO VEÍCULO

In [7]:
limpa_tabela(("veiculo_tipo",))

insert = cur_dest.prep("insert into veiculo_tipo(codigo_tip, descricao_tip) values(?, ?)")

cur_orig.execute("select row_number() over (order by categoria, codigo) codigo, descricao from cgfcategtipo")

for row in cur_orig:
    cur_dest.execute(insert, (row['codigo'], row['descricao']))
commit()

## MARCAS

In [8]:
limpa_tabela(('veiculo_marca',))

insert = cur_dest.prep("insert into veiculo_marca(codigo_mar, descricao_mar) values(?, ?)")

cur_orig.execute("select codigo, nome from cgfmarca c")

for row in cur_orig:
    cur_dest.execute(insert, (row['codigo'], row['nome']))
commit()

## VEÍCULO

In [ ]:
limpa_tabela(('veiculo',))

insert = cur_dest.prep("insert into veiculo(placa, modelo, combustivel, anomod, renavam, chassi, codigo_marca_vei, codigo_tipo_vei, aquisicao, codigo_bem_par, tanque, inativo, sequencia, cor, prefixo) values(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)")

cur_orig.execute("""
select
	a.frota,
	modelo,
	substr(c.descricao,1,1) combustivel,
	substr(anomodelo,1,4) anomodelo,
	b.renavam,
	chassi,
	a.marca,
	d.codigo_categ tipo,
	a.aquisicao::date aquisicao,
	a.codigobem,
	tanque,
	a.ativo,
	row_number() over (order by a.frota) as sequencia,
	a.cor,
    a.prefixo
from
	cgffrota a
join cgfveiculo b on a.frota = b.frota
left join cgfcombustivel c on a.combustivel = c.codigo
left join (select row_number() over (order by categoria, codigo) codigo_categ, categoria, codigo from cgfcategtipo) d on a.tipofrota = d.categoria and b.tipo = d.codigo
""")

for row in cur_orig:
    cur_dest.execute(insert, (row['frota'], row['modelo'], row['combustivel'], row['anomodelo'], row['renavam'], row['chassi'], row['marca'], row['tipo'], row['aquisicao'], row['codigobem'], row['tanque'], row['ativo'], row['sequencia'], row['cor'], row['prefixo']))
commit()